RQ1: Is there an association between repository activity characteristics (number of commits and project age) and the intensity of performance testing configurations (num-
ber of users and test duration) in open-source repositories?


RQ2: Do repositories implementing both GUI and performance end-to-end tests show higher overall project activity (Measured by commits, contributors, total issues, and total pull requests) compared to repositories implementing only one type of test?

In [1]:
!pip install pandas

import pandas as pd

In [ ]:
df_repository = pd.read_csv('E2EGit\\repository.csv')


df_repository = df_repository.rename(columns={'name': 'repository_name'})

# Keep only the columns you want
df_repository = df_repository[['repository_name', 'commits','contributors', 'total_issues', 'total_pull_requests']]

#df_repository = df_repository.drop_duplicates().reset_index(drop=True)  

df_repository.head(10)

C:\Users\malja\AppData\Local\Temp\ipykernel_29336\1023623843.py:1: DtypeWarning: Columns (6,7,8,9,14,15,16,24,25,26,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_repository = pd.read_csv('E2EGit\\repository.csv')


['redhatqe/openshift-python-wrapper' ' ']


In [3]:
numeric_cols = ['commits', 'contributors', 'total_issues', 'total_pull_requests']

for col in numeric_cols:
    df_repository[col] = pd.to_numeric(df_repository[col], errors='coerce')

# Drop rows with NaN (caused by non-numeric values)
df_repository = df_repository.dropna(subset=numeric_cols)

In [ ]:
df_gui_repo_details = pd.read_csv('E2EGit\gui_testing_repo_details.csv')

# Keep only the columns you want
df_gui_repo_details = df_gui_repo_details[['repository_name']]
#df_gui_repo_details = df_gui_repo_details.drop_duplicates().reset_index(drop=True)
df_gui_repo_details.head(10)

,repository_name
0,quantumlib/cirq
1,wandb/weave
2,insightsoftwareconsortium/itk-wasm
3,codyogden/killedbygoogle
4,mattermost/mattermost
5,apache/dolphinscheduler
6,pwa-builder/pwabuilder
7,umputun/remark42
8,apache/deltaspike
9,netflix/conductor


In [ ]:
df_performance_test_details = pd.read_csv('E2EGit\performance_testing_test_details.csv')


# Keep only the columns you want
df_performance_test_details = df_performance_test_details[['repository_name']]
#df_performance_test_details = df_performance_test_details.drop_duplicates().reset_index(drop=True)  
df_performance_test_details.head(10)

,repository_name
0,apache/roller
1,nysenate/openlegislation
2,eclipse/jetty.project
3,apereo/cas
4,mapfish/mapfish-print
5,ippontech/tatami
6,apache/tapestry-5
7,pentaho/pentaho-platform
8,geonetwork/core-geonetwork
9,azure/azure-sdk-for-java


In [6]:
df_performance_gui_merged = pd.merge(df_performance_test_details, df_gui_repo_details, left_on='repository_name', right_on='repository_name', how='inner')

df_performance_gui_merged.head(10)

,repository_name
0,apache/roller
1,apache/tapestry-5
2,zkoss/zkspreadsheet
3,eugenp/tutorials
4,gluufederation/oxauth
5,wso2/product-is
6,eclipse/vorto
7,wso2/product-iots
8,ant-media/ant-media-server
9,kubeshop/testkube


In [7]:
# Not merged repositories (exist in GUI but not in Performance)
not_merged_gui_only = pd.merge(df_gui_repo_details, df_performance_test_details, on='repository_name', how='left', indicator=True)
not_merged_gui_only = not_merged_gui_only[not_merged_gui_only['_merge'] == 'left_only'][['repository_name']]

# Not merged repositories (exist in Performance but not in GUI)
not_merged_perf_only = pd.merge(df_performance_test_details, df_gui_repo_details, on='repository_name', how='left', indicator=True)
not_merged_perf_only = not_merged_perf_only[not_merged_perf_only['_merge'] == 'left_only'][['repository_name']]


# Combine the two "not merged" dataframes
not_merged_all = pd.concat([not_merged_gui_only, not_merged_perf_only]).reset_index(drop=True)


print(not_merged_all)

                                    repository_name
0                                   quantumlib/cirq
1                                       wandb/weave
2                insightsoftwareconsortium/itk-wasm
3                          codyogden/killedbygoogle
4                             mattermost/mattermost
..                                              ...
525                      spring-attic/spring-native
526                                        wso2/apk
527                                apache/eventmesh
528  ballerina-platform/ballerina-performance-cloud
529                             jetty/jetty.project

[530 rows x 1 columns]


In [8]:
# merged repositories (exist in both GUI and Performance) with repository details

repo_both_with_repository_details = pd.merge(df_performance_gui_merged, df_repository, on='repository_name', how='inner')

repo_both_with_repository_details.head(10)

,repository_name,commits,contributors,total_issues,total_pull_requests
0,apache/roller,4755.0,14.0,0.0,128.0
1,apache/tapestry-5,5768.0,37.0,0.0,41.0
2,zkoss/zkspreadsheet,4001.0,11.0,4.0,117.0
3,eugenp/tutorials,15392.0,321.0,733.0,15512.0
4,gluufederation/oxauth,5690.0,50.0,1296.0,575.0
5,wso2/product-is,12891.0,210.0,13272.0,6709.0
6,eclipse/vorto,3031.0,42.0,1341.0,1462.0
7,wso2/product-iots,2451.0,51.0,1353.0,593.0
8,ant-media/ant-media-server,7425.0,34.0,4779.0,1034.0
9,kubeshop/testkube,3133.0,74.0,1902.0,2307.0


In [9]:
# Calculate statistics
mean_values = repo_both_with_repository_details[numeric_cols].mean()
median_values = repo_both_with_repository_details[numeric_cols].median()
std_values = repo_both_with_repository_details[numeric_cols].std()

# Combine into a single dataframe for easy view
stats_both_df = pd.DataFrame({
    'mean': mean_values,
    'median': median_values,
    'std': std_values
})

In [10]:
# not merged repositories (exist in GUI or Performance but not in both) with repository details
repo_not_merged_with_repository_details = pd.merge(not_merged_all, df_repository, on='repository_name', how='inner')

repo_not_merged_with_repository_details.head(10)

,repository_name,commits,contributors,total_issues,total_pull_requests
0,quantumlib/cirq,3756.0,205.0,2191.0,4334.0
1,wandb/weave,2362.0,43.0,22.0,1342.0
2,insightsoftwareconsortium/itk-wasm,3489.0,20.0,227.0,876.0
3,codyogden/killedbygoogle,2013.0,200.0,434.0,1061.0
4,mattermost/mattermost,18316.0,389.0,8073.0,18280.0
5,apache/dolphinscheduler,8267.0,362.0,7157.0,7854.0
6,pwa-builder/pwabuilder,5233.0,88.0,2997.0,1651.0
7,umputun/remark42,3225.0,116.0,624.0,963.0
8,apache/deltaspike,2650.0,62.0,0.0,152.0
9,netflix/conductor,3317.0,233.0,1577.0,1702.0


In [11]:
# Calculate statistics
mean_values = repo_not_merged_with_repository_details[numeric_cols].mean()
median_values = repo_not_merged_with_repository_details[numeric_cols].median()
std_values = repo_not_merged_with_repository_details[numeric_cols].std()

# Combine into a single dataframe for easy view
stats_single_df = pd.DataFrame({
    'mean': mean_values,
    'median': median_values,
    'std': std_values
})

In [12]:
print("Statistics for numeric columns (both repositories):")
print(stats_both_df)

print("Statistics for numeric columns (single repository):")
print(stats_single_df)

Statistics for numeric columns (both repositories):
                            mean  median          std
commits              8203.000000  5055.0  8619.343285
contributors           81.461538    50.0    89.318359
total_issues         2340.538462  1341.0  3563.816461
total_pull_requests  2885.615385  1034.0  4299.641604
Statistics for numeric columns (single repository):
                             mean  median           std
commits              10604.583650  5693.0  13647.976194
contributors           131.568441    79.0    124.344660
total_issues          2695.020913   993.0   8731.818205
total_pull_requests   3841.281369  2010.0   5672.582370


In [13]:
from scipy.stats import zscore
import pandas as pd

# Combine both groups into one dataframe with a label
repo_both_with_repository_details['group'] = 'Both'
repo_not_merged_with_repository_details['group'] = 'Single'
df_all = pd.concat([repo_both_with_repository_details, repo_not_merged_with_repository_details], ignore_index=True)

# Compute z-scores for the numeric columns
for col in numeric_cols:
    df_all[f'{col}_zscore'] = zscore(df_all[col])

# Check average z-scores per group
zscore_means = df_all.groupby('group')[[f'{col}_zscore' for col in numeric_cols]].mean()
print(zscore_means)

        commits_zscore  contributors_zscore  total_issues_zscore  \
group                                                              
Both         -0.173145            -0.395363            -0.040065   
Single        0.004279             0.009771             0.000990   

        total_pull_requests_zscore  
group                               
Both                     -0.165446  
Single                    0.004089  


In [14]:
import numpy as np

def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    pooled_std = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / pooled_std

for col in ['commits', 'contributors', 'total_issues', 'total_pull_requests']:
    d = cohens_d(repo_both_with_repository_details[col], repo_not_merged_with_repository_details[col])
    print(f"{col}: Cohen's d = {d:.3f}")

commits: Cohen's d = -0.177
contributors: Cohen's d = -0.405
total_issues: Cohen's d = -0.041
total_pull_requests: Cohen's d = -0.169
